## importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## loadind the Data

In [ ]:
df = pd.read_csv("../input/banknote-authenticationcsv/BankNote_Authentication.csv")
df.head()

In [ ]:
df.shape

## Checking For Data is Imbalanced Or Not

In [ ]:
df['class'].value_counts()

## Statistical Information

**Running the dataset first loads the data before and then prints summary statistics for each variable.
The values vary with different means and standard deviations, perhaps some normalization or standardization would be required prior to modeling.**

In [ ]:
df.describe()

## Looking Datatypes of the columns

In [ ]:
df.info()

## Looking For Null Values

In [ ]:
df.isnull().sum()

## Exploratoratory Data Analysis

#### As I can see that while pairplot shows all the histogram in which the first two variables have a Gaussian-like distribution and the next two input variables may have a skewed Gaussian distribution or an exponential distribution. 
#### That have some benefit in using a power transform on each variable in order to make the probability distribution less skewed which will likely improve model performance

In [ ]:
sns.pairplot(data=df)

In [ ]:
sns.distplot(df['variance'])

In [ ]:
sns.distplot(df['skewness'])

In [ ]:
sns.distplot(df['curtosis'])

In [ ]:
sns.distplot(df['entropy'])

# Looking for Outliers

In [ ]:
sns.boxplot(y=df['variance'])

In [ ]:
sns.boxplot(y=df['skewness'])

In [ ]:
sns.boxplot(y=df['curtosis'])

In [ ]:
sns.boxplot(y=df['entropy'])

## Splitting the Data into X,y

In [ ]:
X = df.drop('class',axis=1)
y = df['class']

In [ ]:
X.shape[1]

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [ ]:
X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

## Splitting the data into train ,test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [ ]:
y_test

## Model Train Using Neural Network

### Neural Network Learning Dynamics
### I have used a Multilayer Perceptron (MLP) model for the dataset using TensorFlow.

### Given that the dataset is small, a small batch size is probably a good idea, e.g. 16 or 32 rows. Using the Adam version of stochastic gradient descent is a good idea when getting started as it will automatically adapt the learning rate and works well on most datasets.

### Before evaluate models in earnest, it is a good idea to review the learning dynamics and tune the model architecture and learning configuration until we have stable learning dynamics, then look at getting the most out of the model.

### We can do this by using a simple train/test split of the data and review plots of the learning curves. This will help us see if we are over-learning or under-learning; then we can adapt the configuration accordingly.

### First, we must ensure all input variables are floating-point values and encode the target label as integer values 0 and 1.

In [ ]:
import tensorflow as tf

#### Used a minimal MLP model. In this case,  used one hidden layer with 10 nodes and one output layer (chosen arbitrarily) and also the ReLU activation function in the hidden layer and the “he_normal” weight initialization, as together, they are a good practice.

#### The output of the model is a sigmoid activation for binary classification and we will minimize binary cross-entropy loss

In [ ]:
n_features = X.shape[1]
model = tf.keras.Sequential([
tf.keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)),
tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

####  fit the model for 50 training epochs (chosen arbitrarily) with a batch size of 32 because it is a small dataset.

#### As fitting the model on raw data, which might be a good idea, but it is an important starting point.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_test,y_test))

#### At the end of training, evaluated the model’s performance on the test dataset and report performance as the classification accuracy.

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
# predict test set
yhat = model.predict_classes(X_test)
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % score)

## Finally, plotted learning curves of the cross-entropy loss on the train and test sets during training.

In [ ]:
# plot learning curves
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
print(confusion_matrix(y_test,yhat))

In [ ]:
print(classification_report(y_test,yhat))